# Multi-instrument analysis with the SunPy Ecosystem

- AIA
- EUI
- SPICE
- DKIST

Bonus:
- IRIS (?)
- EIS (?)

The affiliated packages we will use are:

- `sunpy`
- `sunpy-soar`
- `dkist`
- `sunraster`

The interesting science question we will answer is...

- Observations from chromosphere (dkist) to TR (SPICE) to corona (AIA, EUI)
- How can additional FOV constraints tell us more about what DKIST is seeing?

In [19]:
from sunpy.net import Fido, attrs as attrs
import dkist.net
import sunpy_soar
import astropy.units as u

from sunpy.coordinates import get_earth, get_horizons_coord

## Searching for and Downloading Data with `Fido`

### SDO

In [5]:
time_range = attrs.Time("2022-10-24T18:30", "2022-10-24T22:20")

In [15]:
aia_query = time_range & attrs.Instrument.aia & attrs.Wavelength(171*u.Angstrom)

In [17]:
q = Fido.search(
    dkist_query
)

In [18]:
q

Start Time,End Time,Instrument,Wavelength,Bounding Box,Dataset ID,Dataset Size,Exposure Time,Primary Experiment ID,Primary Proposal ID,Stokes Parameters,Target Types,Number of Frames,Average Fried Parameter,Embargoed,Downloadable,Has Spectral Axis,Has Temporal Axis,Average Spectral Sampling,Average Spatial Sampling,Average Temporal Sampling
,,,nm,,,Gibyte,s,,,,,,,,,,,nm,arcsec,s
Time,Time,str4,float64[2],str32,str5,float64,float64,str9,str9,str4,str7[1],int64,float64,bool,bool,bool,bool,float64,float64,float64
2022-10-24T19:47:14.651,2022-10-24T20:39:08.319,VISP,630.2424776472172 .. 631.826964866207,"(233.36,-368.86),(202.9,-503.94)",BEOGN,8.0,48.00811267605634,eid_1_123,pid_1_123,IQUV,unknown,4000,6.396623914771764,False,True,True,True,0.00162511509639976,0.04160701928854328,1556.8341594999983
2022-10-24T18:57:45.634,2022-10-24T19:33:26.865,VISP,630.2424776472172 .. 631.826964866207,"(233.36,-368.87),(212.1,-503.97)",BKEWK,5.0,48.00811267605634,eid_1_123,pid_1_123,IQUV,unknown,2752,0.11445240041497136,False,True,True,True,0.00162511509639976,0.04160701928854328,1070.6156795058157


### Solar Orbiter

In [ ]:
eui_query = time_range & attrs.soar.Product('EUI-HRIEUV174-IMAGE') & attrs.Level(2)
spice_query = time_range & attrs.soar.Product('SPICE-N-RAS') & attrs.Level(2)

### DKIST

In [ ]:
dkist_query = time_range & attrs.Instrument('VISP')